In [ ]:
# 5) Include hyper parameter training and exeprimentation
# 6) Start a discussion on the topic 


# MLP Implementation

In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

#import sys
class MLP():

    def __init__(self, numinputs=4, hiddenlayers=[4,2,2,1], numoutputs=1, activationfunc = [1,1,1], alpha = 0.25, lossfunc = 1):
        self.numinputs = numinputs
        self.hiddenlayers = hiddenlayers
        self.numoutputs = numoutputs

        # create a generic representation of the layers
        self.layers = [numinputs] + hiddenlayers + [numoutputs]
         
        # create random connection weights for the layers
        weights = []
        bias = []
        for i in range(len(self.layers)-1):
            bias.append(np.ones(self.layers[i+1]))
            weights.append(np.random.rand(self.layers[i], self.layers[i+1]))
        self.weights = weights
        self.bias = bias
        self.alpha = alpha
        
    
        if lossfunc == 1: 
            self.lss = self.loss_func
        elif lossfunc == 2: 
            self.lss = self.cross_entropy_loss
        elif lossfunc == 3: 
             self.lss = self.binary_cross_entropy_loss
    
        self.activationfun = [np.vectorize(self._sigmoid) if i == 1 else np.vectorize(self._relu) if i == 2   else np.vectorize(self._tanh)   for i in activationfunc]
        self.activationfunderv = [np.vectorize(self._sigmoid_derv) if i == 1 else np.vectorize(self._relu_derv) if i == 2   else np.vectorize(self._tanh_derv)  for i in activationfunc]

# #         #Test
#         self.weights = [np.array([[-0.2, -0.1],
#         [ 0.2,  0.3]]), np.array([[0.2],
#         [0.3]])]
#         self.bias = [np.array([0.1, 0.1]), np.array([0.2])]
        
        
        
    def transform_vector(self,weights_trial):
        weights = []
        index = 0
        for i in range(len(self.layers)-1):
            count = self.layers[i]*self.layers[i+1]
            weights.append(np.array(weights_trial[index:count+index]).reshape(self.layers[i+1],self.layers[i]).transpose())
            index+=count
        self.weights = weights
       
        return 1 
    


    def weight_update(self,weight_update):
        self.weights = [  w-w_u for w,w_u in zip(self.weights, weight_update) ]
        
    
    def bias_update(self,bias_update): 
        self.bias = [  b-b_u for b,b_u in zip(self.bias, bias_update) ]

    
    def _sigmoid(self, x):
        return 1.0 / (1 + np.exp(-x))
        
    
    def _relu(self, x): 
        return max(0, x)
    
    def _tanh(self, x): 
         return (np.exp(x) - np.exp(-x))/ (np.exp(x)+ np.exp(-x))
        
        
    def _sigmoid_derv(self, x): 
             val = self._sigmoid(x) 
             return val * (1-val)
       

    def _relu_derv(self, x): 
        return 1 if x>0 else 0
    
    def _tanh_derv(self, x):
        return 1 - (((np.exp(x) - np.exp(-x))**2)/ ((np.exp(x)+ np.exp(-x))**2))

    def loss_func(self, x,y): 
        #add aquare root
        fun = np.vectorize(math.sqrt)
        return (y-x)**2
    
    
    def cross_entropy_loss(self,pred, y):
        if y == 1:
            return -np.log(pred)
        else:
            return -np.log(1 - pred)
        
    def binary_cross_entropy_loss(self,pred,y): 
        return -((y*np.log(pred)) + (1-y) * np.log(1-pred))
  

    def apply_loss(x,y): 
        return self.lss(x,y)
        
    def forward_propagate(self,inputs):
        activations = inputs
        net_inputs = []
        z_value = []
        activated_value = []
        activated_value.append(inputs)
       
        for w,b,a in zip(self.weights,self.bias,self.activationfun):
            net_inputs = (np.dot(activations, w)) + b
            activations = a(net_inputs)
            z_value.append(net_inputs)
            activated_value.append(activations)

        return z_value, activated_value
    

    def back_propogate(self,z_value, activated_value,  y,step):
        eval =  self.lss(activated_value[-1],y)
        weight_update = []
        bias_update = []
        sig_derv_activ = []
        for i in range(len(self.weights)-1, -1,-1):
                
                sig_derv_activ = self.activationfunderv[i](z_value[i])*eval
                weight_updt = self.alpha* (1.0/step)*np.sum(np.array([np.dot(np.reshape(sig_derv_activ[x],(-1,1)),np.reshape(activated_value[i][x],(-1,1)).transpose()) for x in range((len(activated_value[i])))]),axis=0)

                bias_updt = np.average(sig_derv_activ,axis = 0)* self.alpha
                eval = [np.dot(np.reshape(sig_derv_activ[x], (-1, 1)).transpose(),self.weights[i].transpose()) [0]for x in range(len(sig_derv_activ))]
                weight_update.insert(0,weight_updt.transpose())
                bias_update.insert(0,bias_updt)
        return weight_update, bias_update
        
        
        
    def predict_values(self, inputs, y): 
            z_value, activated_value = self.forward_propagate(np.array(inputs))
            
            loss = self.lss(activated_value[-1],np.reshape(np.array(y),(-1,1)))
            cost = (1.0/len(y))*sum(loss)

            return  cost
        
        

def training(df_X,hiddenlayers,numoutputs,activationfunc,alpha,loss,step, epochs,Y_train):
    print('Activation Functions are as follows: 1 -> Sigmoid | 2-> ReLu | 3-> tanH')
    print('Loss Functions are as follows: 1 -> Error | 2-> Cross Entropy')
    print('#########')
    print('Results for hyperparameter:')
    print('Epochs:' +str(epochs)) 
    print('Steps:' + str(step))
    print('Learning rate:' +str(alpha))
    print('Number of hidden layers:'+str(len(hiddenlayers)) )
    print('Loss function:'+str(lossfunc))
    print('Activation functions:'+str(activationfunc))
    mlp = MLP(df_X.shape[1], hiddenlayers, numoutputs,activationfunc, alpha, lossfunc)

     # Need to add this as arg as well and epochs
    for i in range(epochs):
        for i in range(1, sample_len,step): #sample_len,5): 
            z_value, activated_value =mlp.forward_propagate(np.array(X_train.iloc[0:i]))
            weight_update, bias_update= mlp.back_propogate(z_value,activated_value, np.array(Y_train.iloc[0:i]).reshape(i,1),step)
            mlp.weight_update(weight_update)
            mlp.bias_update(bias_update)
    return mlp
           
    
# if __name__ == "__main__":

# #     df = pd.read_csv('/Users/smiroshnikova/Desktop/trial.csv',header=None)
#     df = pd.read_csv('data_banknote_authentication.csv',header=None)

#     df_Y = df.iloc[:,-1]
#     df_X = df.iloc[:,0:-1]
#     X_train,X_test,Y_train,Y_test=train_test_split(df_X,df_Y,test_size=0.15)
#     sample_len = len(X_train)
#     epochs = 1
#     hiddenlayers=[2,2]
#     numoutputs=1
#     activationfunc = [1,1,1]
#     alpha = 0.2
#     lossfunc =3
#     step = 5 
#     epochs = 500

    

   

# PSO for MLP Implementation

In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import random
from sympy.utilities.iterables import multiset_permutations
import itertools    
import random  



class particle(): 
    def __init__(self,x):
        self.velocity = []
        self.position = x
        self.best_position = []
        self.best_error = -1
        self.error = -1
        self.inf_best_error = -1
        self.inf_best_position = []
        self.w=0.5 
        self.num_dimensions = len(x)
        for i in range(0,self.num_dimensions):
            self.velocity.append(random.uniform(-1,1))
        
            

    
    def evaluate(self,costFunc):
        
        self.err=costFunc(self.position)
        if  self.best_error==-1 or self.error < self.best_error:
            self.pos_best=self.position
            self.best_error=self.error
    
    def update_velocity(self,post_best_g): 
        for i in range(0,self.num_dimensions):
            r1=random.random()
            r2=random.random()
            r3=random.random()

            vel_personal=r1*(self.pos_best[i]-self.position[i])
            vel_global=r2*(post_best_g[i]-self.position[i])
            vel_informant=r3*(self.inf_best_position[i])-self.position[i]
            self.velocity[i]=self.w*self.velocity[i]+vel_personal+vel_global+vel_informant
        
    
    def update_position(self): 
           for i in range(0,self.num_dimensions):
                self.position[i]=self.position[i]+self.velocity[i]
                
        
class PSO():
    def __init__(self,costFunc,samples,num_particles,max_iterations,max_informants):
        num_dimensions=len(samples[0])
        err_best_g=-1                   
        pos_best_g=[]                  

        swarm=[]
        swarm_informants=[]
       
    

        for i in range(0,num_particles):
            swarm.append(particle(samples[i]))
    
        for i in range(0,num_particles): 
            swarm_informants.append(random.sample(swarm[0:i]+swarm[i+1:num_particles],max_informants))
        
        
        for i in range(0,max_iterations):
#             print('Iteration no:' + str(i))
            
            for j in range(0,num_particles):
#                 print('Particle no:' + str(j))
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if  ( err_best_g == -1 or swarm[j].err < err_best_g ) and not math.isnan(err_best_g)  :
                    
                    pos_best_g=list(swarm[j].position)
                    err_best_g=float(swarm[j].err)
                    print(err_best_g)
                    
                
                # determine the best amongst the informants for each particle
                for x in range(0,max_informants):
                    if swarm[j].inf_best_error == -1 or swarm[j].inf_best_error>swarm_informants[x].err :
                        swarm[j].inf_best_error=float(swarm_informants[j][x].error)
                        swarm[j].inf_best_position=list(swarm_informants[j][x].position)
                    

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position()
 

        # print final results
        print 'FINAL:'
        print pos_best_g
        print err_best_g
       
        
        




In [3]:
def generate_weight(layers):
    weight = []
    for i in range(len(layers)-1):
            x = np.random.rand(layers[i], layers[i+1])
            weight.extend(np.concatenate(x.transpose()))
    return weight
    




In [ ]:
df = pd.read_csv('/Users/smiroshnikova/Desktop/CW1/data_banknote_authentication.csv',header=None)

df_Y = df.iloc[:,-1]
df_X = df.iloc[:,0:-1]
X_train,X_test,Y_train,Y_test=train_test_split(df_X,df_Y,test_size=0.15)
    
hiddenlayers=[2,3,2]
numoutputs=1
activationfunc = [1,1,1,1]
alpha = 0.25
lossfunc =3
step = 5 
epochs = 500
num_particles = 50
max_iterations = 500
max_informants = 6
numinputs = X_train.shape[1]
sample_len = 5

weights = []
for i in range(num_particles):
    weights.append(generate_weight([numinputs] + hiddenlayers + [numoutputs]))
    
def cost_func(x):
    mlp = MLP(numinputs, hiddenlayers, numoutputs,activationfunc , alpha, lossfunc)

    mlp.transform_vector(x)
    
    cost = mlp.predict_values(X_train,Y_train)
    return cost

import time


print('Executing PSO')
print('--------------------')
start_time = time.time()
PSO(cost_func,weights,num_particles,max_iterations,max_informants)
print("--- %s seconds ---" % (time.time() - start_time))

print('Executing MLP')
print('--------------------')
mlp = training(X_train, hiddenlayers, numoutputs,activationfunc , alpha, lossfunc, step, epochs,Y_train)
cost = mlp.predict_values(X_test,Y_test)
print("--- %s seconds ---" % (time.time() - start_time))


Executing PSO
--------------------


# Hyperparameter tunning for MLP

In [ ]:
epoch_test = []
for i in range(0, 10000, 2500): 
        mlp = training(X_train, hiddenlayers, numoutputs,activationfunc , alpha, lossfunc, step, i,Y_train)
        pred_val,loss, cost = mlp.predict_values(X_test,Y_test)
        epoch_test.append((i,cost))
  


In [ ]:
steps_test = []
for i in range(1,len(X_train), 20): 
        mlp = training(X_train, hiddenlayers, numoutputs,activationfunc , alpha, lossfunc, i, epochs,Y_train)
        pred_val,loss, cost = mlp.predict_values(X_test,Y_test)
        steps_test.append((i,cost))
        
    

In [ ]:
activation_static_test = []
for i in range(1,4):
        mlp = training(X_train, hiddenlayers, numoutputs,[i for x in range(len(hiddenlayers) + 1)], alpha, lossfunc, step, epochs,Y_train)
        pred_val,loss, cost = mlp.predict_values(X_test,Y_test)
        activation_static_test.append((i,cost))
        
    

In [ ]:
alpha_test = []
for i in np.arange(0,1, 0.10):
        mlp = training(X_train, hiddenlayers, numoutputs,activationfunc , i, lossfunc, step, epochs,Y_train)
        pred_val,loss, cost = mlp.predict_values(X_test,Y_test)
        alpha_test.append((i,cost))
    
   

In [ ]:
lossfunc_test = []
 
for i in range(1,3):
        mlp = training(X_train, hiddenlayers, numoutputs,activationfunc , alpha, i, step, epochs,Y_train)
        pred_val,loss, cost = mlp.predict_values(X_test,Y_test)
        lossfunc_test.append((i,cost))


In [ ]:

    from sympy.utilities.iterables import multiset_permutations
    import numpy as np

    perms = []
    perms_test = []
    for i in range(1,6): 

        for p in multiset_permutations(np.random.permutation(i)+1):
            perms.append(p)

    for i in perms: 
        
        mlp = training(X_train, i, numoutputs,[1]*(len(i)+1) , alpha, lossfunc, step, epochs,Y_train)
        pred_val,loss, cost = mlp.predict_values(X_test,Y_test)
        perms_test.append((i,cost))